In [108]:
import pandas as pd
import numpy as np
import datasets
from inference_utils import *
import json
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import utils
from utils import  get_stats

### read jsonl file
aspect = 'all'

data = datasets.load_dataset('boda/review_evaluation_automatic_labels', aspect)['test']

gold_data = get_gold_labels(data, aspect,'chatgpt_ASPECT_score')





In [109]:

model_outputs = []
parent_dir = 'evalute_outputs/full_model/scilitllm/score_only/instruction/all/label_loss/'
data_path =   parent_dir + 'predictions.jsonl'


with open(data_path) as f:
    for line in f:
        model_outputs.append(json.loads(line))
        

In [110]:
assert len(model_outputs) == len(gold_data)

In [111]:
aspect

'all'

In [112]:
aspects = [ 'actionability', 'grounding_specificity','verifiability', 'helpfulness'] if aspect == 'all' else [aspect]
## convert the list of dictionaries to dictionary of lists


predctions = {aspect:[] for aspect in aspects}
labels = {aspect:[] for aspect in aspects}

for i in range(len(model_outputs)):
    for aspect in aspects:

        l = gold_data[i][aspect]
        p = model_outputs[i][f'{aspect}_label']
        l = str(l)
        p = str(p)

        if p == 'None':
            continue
        predctions[aspect].append(p)
        labels[aspect].append(l)

In [113]:
## write the stats into file
with open(f'{parent_dir}evalute_outputs.txt', 'w') as f:

    f.write('number of samples: ' + str(len(labels[aspect])) + '\n')
    for a in aspects:
        stat_dict = get_stats(predctions[a], labels[a], a)
        f.write('*' * 20 + f'{a}'+ '*' * 20 + '\n')

        ## remove the acc from the dict, and for spearman, only include the correlation

        for k,v in stat_dict.items():
            if 'accuracy' in k:
                continue
            if 'spearman' in k:
                v = v[0]
            ## round the values
            if isinstance(v, float):
                v = round(v, 3)
            f.write(f'{k}: {v}\n')
        f.write('-'*50+'\n')



In [114]:
# data = datasets.load_dataset('boda/review_evaluation_automatic_labels', 'all')




In [115]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Convert score values to numeric before computing value counts
# value_counts = pd.DataFrame({
#     aspect: pd.to_numeric(data['train'].to_pandas()[f'chatgpt_{aspect}_score'], errors='coerce').value_counts()
#     for aspect in aspects
# })

# # Sort index to ensure correct order
# value_counts = value_counts.sort_index()

# # Plot the combined value counts
# value_counts.plot(kind='bar', title='Aspect Distribution')
# plt.xlabel('Scores')
# plt.ylabel('Counts')
# plt.show()


In [116]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/scitulu-7b")

def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids))[0:])

prompt = """###Review Point:
3. The theoretical results are based on certain assumptions that may not hold in all practical scenarios. 
###Output:
{'actionability_label':"""
print_tokens_with_ids(prompt)  # [..., ('▁Hello', 15043), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901), ...]

response_template = "\n\n###Output:\n"
print_tokens_with_ids(response_template)  # [('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

tokenizer(response_template)['input_ids'][2:]

[('▁###', 835), ('Re', 1123), ('view', 1493), ('▁Point', 8984), (':', 29901), ('<0x0A>', 13), ('3', 29941), ('.', 29889), ('▁The', 450), ('▁theoretical', 23399), ('▁results', 2582), ('▁are', 526), ('▁based', 2729), ('▁on', 373), ('▁certain', 3058), ('▁assumptions', 20813), ('▁that', 393), ('▁may', 1122), ('▁not', 451), ('▁hold', 4808), ('▁in', 297), ('▁all', 599), ('▁practical', 15031), ('▁scenarios', 21846), ('.', 29889), ('▁', 29871), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('Output', 6466), (':', 29901), ('<0x0A>', 13), ("{'", 10998), ('action', 2467), ('ability', 3097), ('_', 29918), ('label', 1643), ("':", 2396)]
[('▁', 29871), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('Output', 6466), (':', 29901), ('<0x0A>', 13)]


[13, 13, 2277, 29937, 6466, 29901, 13]